# This is the code for Kawasaki disease (KD) and Normal binary classification
## Package requirments:
**scikit-learn, matplotlib, pandas,torch and joblib**
You can install it using pip. 
For example: pip install scikit-learn
## \*First step is to preprocess the data so we will have selected features and the adjusted tabular data
## \*Second step is to loading the model and run the classification

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torch
import torch.nn as nn
import random
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score
from sklearn.metrics import auc,classification_report,RocCurveDisplay
import joblib

In [8]:
def resize_data(df, constant_features,common_features):
    df_c = df[constant_features+common_features]
    # Initial feature dictionary
    feature_name = []
    name_list = ['II','III','V1','V2','V3','V4','V5','V6','aVF','aVL','aVR']
    for num in range(11):
        for feature in common_features:
            feature_name.append(feature+'_'+name_list[num])
    #print(feature_name)

    # row goes to the column feature
    df_c_0 = df_c.iloc[0:len(df_c):12]
    features_len = len(common_features)
    for i in range(1,12):
        temp = df_c.iloc[i:len(df_c):12,len(constant_features):]
        temp.set_axis(feature_name[(i-1)*features_len:i*features_len], axis=1, copy=False)
        df_c_0 = pd.concat([df_c_0.reset_index(drop=True),temp.reset_index(drop=True)], axis=1)
    return df_c_0.reset_index(drop=True)

def data_preprocessing(test_df):#selecting the features
    constant_features = ['QRS_Duration__ms_C','R_T_Axis_deviation','Group','Gender','Ventricular_Rate_C', 'P_Axis_C', 'R_Axis_C', 'T_Axis_C',
       'QTc_Calculation_C', 'QTc_Bazett_C', 'QTc_Friderica_C', 'PR_Interval_C','RR_Interval_C', 'Q_T_Interval_C']
    print(len(constant_features))
    common_features = ['Q_Duration_C','R_Duration_C',
       'S_Duration_C', 'P_Peak_Amplitude_C',
       'Q_Peak_Amplitude_C', 'R_Peak_Amplitude_C', 'S_Peak_Amplitude_C',
       'T_Peak_Amplitude_C', 'Q_Area_C', 'R_Area_C', 'S_Area_C', 'T_Area_C',
       'Summated_QRS_integral']
    label = test_df['KD'].iloc[0:len(test_df):12].values
    df_normal_features = constant_features+common_features
    test_df = test_df[df_normal_features].copy()
    qt = test_df['Q_T_Interval_C'].values
    rr = test_df['RR_Interval_C'].values

    qtc_f  = qt/(rr**(1/3))*10
    qtc_f = [(round(n)) for n in qtc_f]
    test_df['QTc_Friderica_C'] = qtc_f

    #qtc_b  = qt/(rr**(1/2))*10
    qtc_b  = qt/((rr/1000)**(1/2))
    qtc_b = [(round(n)) for n in qtc_b]
    test_df['QTc_Bazett_C'] = qtc_b
    
    df_resize = resize_data(test_df, constant_features,common_features)
    rename_dic = {}
    for name in common_features:
        rename_dic[name] = name+'_I'
    df_resize = df_resize.rename(columns=rename_dic)
    df_resize['Gender'] = df_resize['Gender'].replace(['Female', 'Male'], [0, 1])

    #df_resize['Gender'] = df_resize['Gender'].replace({'Female': 0, 'Male': 1})
    #df_resize['Gender'].replace(['Female', 'Male'],[0, 1], inplace=True)


    return label,df_resize



## Step 1 preprocessing the data

* 12 leads will be extended to long features with one raw
* resacale the feature **QTc_Friderica_C** and **QTc_Bazett_C**

In [9]:
KD_file_path_data =  './data/data_test_kd.csv'
KD_raw_data = pd.read_csv(KD_file_path_data, sep = ',')
y_KD,df_KD = data_preprocessing(KD_raw_data)
df_KD

14


/tmp/ipykernel_93981/1298472573.py:49: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_resize['Gender'] = df_resize['Gender'].replace(['Female', 'Male'], [0, 1])


,QRS_Duration__ms_C,R_T_Axis_deviation,Group,Gender,Ventricular_Rate_C,P_Axis_C,R_Axis_C,T_Axis_C,QTc_Calculation_C,QTc_Bazett_C,...,P_Peak_Amplitude_C_I,Q_Peak_Amplitude_C_I,R_Peak_Amplitude_C_I,S_Peak_Amplitude_C_I,T_Peak_Amplitude_C_I,Q_Area_C_I,R_Area_C_I,S_Area_C_I,T_Area_C_I,Summated_QRS_integral_I
0,68.0,40.0,12.0,0,106.0,14.0,83.0,43.0,422.0,422,...,-83.0,0.0,87.0,986.0,-229.0,0.0,72.0,1934.0,-2339.0,2006.0
1,72.0,47.0,12.0,0,128.0,22.0,76.0,29.0,414.0,414,...,-102.0,756.0,117.0,0.0,-214.0,1486.0,102.0,0.0,-2360.0,1588.0
2,76.0,4.0,11.0,1,103.0,53.0,26.0,22.0,442.0,444,...,-92.0,0.0,34.0,498.0,-92.0,0.0,22.0,823.0,-1303.0,845.0
3,86.0,29.0,11.0,1,78.0,48.0,70.0,41.0,435.0,436,...,-92.0,639.0,190.0,0.0,-449.0,489.0,138.0,0.0,-2525.0,627.0
4,82.0,21.0,11.0,1,96.0,70.0,74.0,53.0,412.0,411,...,-102.0,893.0,156.0,0.0,-258.0,684.0,147.0,0.0,-1437.0,831.0


In [10]:
Normal_file_path_data =  './data/data_test_normal.csv'
Normal_raw_data = pd.read_csv(Normal_file_path_data, sep = ',')
y_Normal ,df_Normal = data_preprocessing(Normal_raw_data)
df_Normal

14


/tmp/ipykernel_93981/1298472573.py:49: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_resize['Gender'] = df_resize['Gender'].replace(['Female', 'Male'], [0, 1])


,QRS_Duration__ms_C,R_T_Axis_deviation,Group,Gender,Ventricular_Rate_C,P_Axis_C,R_Axis_C,T_Axis_C,QTc_Calculation_C,QTc_Bazett_C,...,P_Peak_Amplitude_C_I,Q_Peak_Amplitude_C_I,R_Peak_Amplitude_C_I,S_Peak_Amplitude_C_I,T_Peak_Amplitude_C_I,Q_Area_C_I,R_Area_C_I,S_Area_C_I,T_Area_C_I,Summated_QRS_integral_I
0,64.0,49.0,6.0,1,113.0,79.0,78.0,29.0,422.0,422,...,-78.0,0.0,34.0,825.0,-117.0,0.0,28.0,1005.0,-1134.0,1033.0
1,56.0,27.0,8.0,1,128.0,72.0,35.0,8.0,408.0,410,...,-87.0,322.0,122.0,0.0,-126.0,381.0,132.0,0.0,-1257.0,513.0
2,90.0,13.0,10.0,0,85.0,2.0,54.0,41.0,402.0,403,...,-112.0,0.0,24.0,874.0,-336.0,0.0,20.0,1506.0,-2797.0,1526.0
3,82.0,14.0,12.0,0,81.0,43.0,51.0,37.0,427.0,428,...,-112.0,0.0,63.0,1074.0,-434.0,0.0,24.0,909.0,-2326.0,933.0
4,88.0,11.0,12.0,1,92.0,59.0,60.0,49.0,417.0,419,...,-83.0,590.0,175.0,0.0,-346.0,412.0,175.0,0.0,-1691.0,587.0


In [11]:
combined_df = pd.concat([df_KD.assign(label=1), df_Normal.assign(label=0)]).sample(frac=1).reset_index(drop=True)
X_train = combined_df.iloc[:, :-1].values
scaler = joblib.load('transformer_KD_scaler.joblib')
X_train = scaler.transform(X_train)

features = torch.tensor(X_train, dtype=torch.float)
labels = torch.tensor(combined_df['label'].values, dtype=torch.long)
TensorDataset(features, labels)
test_loader = DataLoader(TensorDataset(features, labels), batch_size=10, shuffle=False, drop_last=False)


## Step 2 Loading the tranning model

## Transformer based method

In [12]:

seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)
random.seed(seed)
class TransformerModel(torch.nn.Module):
    def __init__(self, num_features, num_classes, dim_model, num_heads, num_layers, dropout_rate):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(num_features, dim_model)  # Project input features to dim_model
        self.transformer_layer = torch.nn.TransformerEncoderLayer(
            d_model=dim_model,   # Use dim_model here
            nhead=num_heads,
            dropout=dropout_rate
        )
        self.encoder = torch.nn.TransformerEncoder(self.transformer_layer, num_layers=num_layers)
        self.output_layer = nn.Linear(dim_model, 1)  # Match dim_model with output_layer input size

    def forward(self, src):
        src = self.embedding(src)  # Project the input features to the model dimension[batch_size, 1, feature_dim]
        #print('src1',src.shape)
        src = src.unsqueeze(1)
        #src = src.permute(1, 0, 2)  # Rearrange to [seq_len, batch_size, feature_dim]
        #print('src2',src.shape)
        x = self.encoder(src)
        x = torch.mean(x, dim=1)  # Mean pooling over the sequence length
        x = self.output_layer(x)
        x = torch.sigmoid(x)#(x, dim=1)
        return x

model = TransformerModel(num_features=170, num_classes=2, dim_model=64, num_heads=2, num_layers=4, dropout_rate=0.2)
model_save_path = "./transformer_model.pth"
checkpoint = torch.load(model_save_path)
model.load_state_dict(checkpoint['model_state_dict'])


/home/honggen/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


<All keys matched successfully>

In [13]:
for features, labels in test_loader:
    outputs = model(features)
    prob = outputs.squeeze()
    predicted = (prob > 0.5).float()
    #_, predicted = torch.max(outputs, dim=1)
    print('true:',labels)
    print('predict:',predicted)
    #loss = criterion(outputs, labels)

true: tensor([0, 1, 0, 1, 0, 0, 1, 1, 1, 0])
predict: tensor([0., 1., 1., 1., 0., 1., 0., 0., 1., 0.])


In [14]:
tn, fp, fn, tp = confusion_matrix(labels,predicted).ravel()
print('true negative (normal): ', tn/(tn+fp))
print('true positive (disease): ',tp/(tp+fn))

true negative (normal):  0.6
true positive (disease):  0.6
